In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout, Permute, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import History
import keras_tuner
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import KFold
from kerastuner import HyperModel, RandomSearch, Hyperband
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import LeakyReLU, ReLU
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
y = np.load('combined_labels.npy')
X = np.load('combined_features.npy')
y = y.reshape(-1, 1)

In [ ]:
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y)
rows_to_remove = [87, 88, 89, 90]
X_input = np.delete(X, rows_to_remove, axis=0)
X_predictions = X[87:91]
y_input = np.delete(y_scaled, rows_to_remove, axis=0)
y_predictions = y_scaled[87:91]
X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=0.3, random_state=42)

In [ ]:
def r_squared(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())

In [ ]:
def create_cnn_model(input_shape, initial_learning_rate, total_steps):
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_learning_rate,  
        decay_steps=total_steps,  
        alpha=0.0 
    )
    model = Sequential([
        Input(shape=input_shape),
        Permute((2, 1)),
        Conv1D(36, kernel_size=3,padding='same', kernel_regularizer = regularizers.l1(3e-5)),
        LeakyReLU(negative_slope=0.2),
        MaxPooling1D(pool_size=2),
        Conv1D(56, kernel_size=3,padding='same', kernel_regularizer = regularizers.l1(1e-4)),
        LeakyReLU(negative_slope=0.2),
        MaxPooling1D(pool_size=2),
        Conv1D(36, kernel_size=3, padding='same', kernel_regularizer=regularizers.l1(6e-3)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(256, activation='relu', kernel_regularizer=regularizers.l2(2e-3)),
        Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),
        Dense(160, activation='relu', kernel_regularizer=regularizers.l1(4e-3)),
        Dense(1)
    ])

    optimizer = Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae', r_squared])
    return model

In [ ]:
def create_cnn_lstm_model(input_shape, initial_learning_rate, total_steps):
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_learning_rate,  
        decay_steps=total_steps,  
        alpha=0.0 
    )
    model = Sequential([
        Input(shape=input_shape),
        Permute((2, 1)),
        Conv1D(44, kernel_size=3, padding='same', kernel_regularizer = regularizers.l2(2e-5)),
        LeakyReLU(negative_slope=0.2),
        MaxPooling1D(pool_size=2),
        Permute((2, 1)),
        LSTM(72, return_sequences=True, kernel_regularizer=regularizers.l1(6e-3)),
        Dropout(0.2),
        LSTM(32, return_sequences=True, kernel_regularizer=regularizers.l1(7e-5)),  
        Dropout(0.2),
        LSTM(56, return_sequences=False, kernel_regularizer=regularizers.l1(8e-4)),  
        Dropout(0.4),
        Flatten(),
        Dense(192, activation='relu', kernel_regularizer=regularizers.l1(1e-3)),
        Dense(1)
    ])

    optimizer = Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae', r_squared])
    return model


In [ ]:
def create_lstm_model(input_shape, initial_learning_rate, total_steps):
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=initial_learning_rate,  
        decay_steps=total_steps, 
        alpha=0.0 
    )
    model = Sequential([
        Input(shape=input_shape),
        LSTM(16, return_sequences=True, kernel_regularizer=regularizers.l1(1e-4)),
        Dropout(0.3),
        LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(4e-4)),  
        Dropout(0.3),
        LSTM(88, return_sequences=False, kernel_regularizer=regularizers.l1(2e-4)),  
        Dropout(0.2),
        Flatten(),
        Dense(96, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        Dense(1)
    ])

    optimizer = Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae', r_squared])
    return model

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2])
total_steps = (len(X_train) // 64) * 100
model1 = create_cnn_lstm_model(input_shape, initial_learning_rate=0.005, total_steps=total_steps)
history1 = History()
model1.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[history])

model2 = create_cnn_model(input_shape, initial_learning_rate=0.002, total_steps=total_steps)
history2 = History()
model2.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[history])

model3 = create_lstm_model(input_shape, initial_learning_rate=0.001, total_steps=total_steps)
history3 = History()
model3.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[history])